In [27]:
import pandas as pd 
import json
import glob, re

In [13]:
ls ../data/interleaving_results/v3_interleaving_results/

iter_0_SFR_V3_10000_10280.json  iter_3_SFR_V3_10000_10280.json
iter_1_SFR_V3_10000_10280.json  iter_4_SFR_V3_10000_10280.json
iter_2_SFR_V3_10000_10280.json  iter_5_SFR_V3_10000_10280.json


In [31]:
results = glob.glob('../data/interleaving_results/v3_interleaving_results/*')
results = sorted(results, key=lambda x: int(re.search(r'iter_(\d)', x)[1]))

In [55]:
all_query_results = []
for i, fname in enumerate(results):
    query_result = pd.DataFrame(json.load(open(fname)))
    query_result = query_result.rename(columns={
        'query': f'query_{i+1}',
        'dr_sources': f'retrievals_{i+1}'
    })
    if i >0:
        query_result = query_result.drop(columns='url')
    all_query_results.append(query_result)

In [81]:
ground_truth = pd.DataFrame(json.load(open('../data/v3_sources/v3_combined_ALL_with_oracle.json')))

In [90]:
all_query_results_df = pd.concat(all_query_results, axis=1)
all_query_results_df = (
    all_query_results_df
         .merge(ground_truth[['url', 'truth']])
)

In [93]:
# all_query_results_df.to_json('../data/interleaving_results/stitched_sample_with_ground_truth.jsonl.gz',lines=True, compression='gzip', orient='records')

In [91]:
query_cols = list(filter(lambda x: 'query' in x, all_query_results_df.columns))

In [104]:
idx = 2
all_query_results_df[query_cols].iloc[idx].to_dict()

{'query_1': 'What are the most effective strategies for individuals to prepare for and navigate a potential recession, from a financial expert who has built a successful business and wealth through her own entrepreneurial endeavors?',
 'query_2': ' What strategies have been effective for small business owners or entrepreneurs in countries that have recently experienced a recession, such as Greece or Argentina, in terms of preparing for and navigating economic downturns?',
 'query_3': ' What are the most valuable lessons learned and strategies employed by individuals who have successfully navigated a recession in the past, and how can their experiences inform our preparation for a potential future recession?',
 'query_4': ' What specific strategies have individuals from low-income households or economically disadvantaged regions used to prepare for and navigate past recessions, and what lessons can be learned from their experiences?',
 'query_5': ' What strategies have individuals with 

In [105]:
pd.DataFrame(all_query_results_df['truth'].iloc[idx])[['Name', 'Biography', 'Information', 'Perspective', 'Centrality', 'llama_label']]

,Name,Biography,Information,Perspective,Centrality,llama_label
0,Ann Kaplan,A business Ph.D. and fintech entrepreneur who ...,Ann Kaplan shares her top 18 recession money r...,Authoritative,High,Expert
1,Catherine Valega,A CFP and wealth consultant,Catherine Valega suggests that workers aim for...,Informative,Medium,Expert
2,Equifax,A credit reporting agency,Equifax states that credit scores from 580 to ...,Informative,Low,Data Resource
3,Jim Cramer,A CNBC personality,Jim Cramer advises buying tangible stocks duri...,Supportive,Medium,Expert
